In [1]:
!pip install opencv-python-headless
!pip install pandas

In [2]:
from __future__ import annotations
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import label

In [17]:
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import label

def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    return float((img < dark_thresh).sum()) / img.size

def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)

def analyse_folder_darkness(folder: str, dark_thresh: int = 40, csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df

def preprocess(img: np.ndarray) -> np.ndarray:
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    img = cv2.convertScaleAbs(img, alpha=1.5, beta=0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(img)

def tune_params(tile: np.ndarray, low: float = 0.40, mid: float = 0.20, area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:
        return area_min_urban / 5, 1.5
    if dr > mid:
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2

def merge_boxes(boxes: List[Tuple[int, int, int, int]], merge_distance: int = 10) -> List[Tuple[int, int, int, int]]:
    if not boxes:
        return []
    boxes = [list(b) for b in boxes]
    merged = True
    while merged:
        merged = False
        result = []
        used = [False] * len(boxes)
        for i in range(len(boxes)):
            if used[i]: continue
            x1, y1, x2, y2 = boxes[i]
            for j in range(i + 1, len(boxes)):
                if used[j]: continue
                x1_, y1_, x2_, y2_ = boxes[j]
                if x1_ <= x2 + merge_distance and x2_ >= x1 - merge_distance and \
                   y1_ <= y2 + merge_distance and y2_ >= y1 - merge_distance:
                    x1 = min(x1, x1_); y1 = min(y1, y1_); x2 = max(x2, x2_); y2 = max(y2, y2_)
                    used[j] = True
                    merged = True
            result.append((x1, y1, x2, y2))
            used[i] = True
        boxes = result
    return boxes

def detect_objects(tile: np.ndarray, area_thresh: float, intensity_ratio: float) -> List[Tuple[int, int, int, int]]:
    img = preprocess(tile)
    bin_img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objs = []
    h, w = tile.shape
    bg_mean = np.median(tile)

    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh

        if area < area_thresh * h * w:
            continue

        roi = tile[y:y+bh, x:x+bw]
        if roi.mean() < bg_mean * intensity_ratio:
            continue

        aspect = max(bw / bh, bh / bw)
        if aspect > 8:
            continue

        compactness = area / (4 * np.pi * ((bw / 2 + bh / 2)**2))
        if compactness < 0.02:
            continue

        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs)


def segment_water_land(img: np.ndarray, water_thresh_factor: float = 0.6) -> np.ndarray:
    blurred = cv2.medianBlur(img, 5)
    median_intensity = np.median(blurred)
    adaptive_thresh = int(median_intensity * water_thresh_factor)
    _, mask = cv2.threshold(blurred, adaptive_thresh, 255, cv2.THRESH_BINARY_INV)
    return mask

def draw_water_contours(image: np.ndarray, water_mask: np.ndarray) -> np.ndarray:
    contours, _ = cv2.findContours(water_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    output = image.copy()
    cv2.drawContours(output, contours, -1, (255, 0, 0), 2)
    return output

def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles

def process_image(image_path: str, max_tile_size=(2000, 2000), low: float = 0.40, mid: float = 0.20, area_min: float = 0.001):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    boxes = []
    for tile, xo, yo in split_image(img, max_tile_size):
        area_t, ratio_t = tune_params(tile, low, mid, area_min)
        for x1, y1, x2, y2 in detect_objects(tile, area_t, ratio_t):
            boxes.append((x1 + xo, y1 + yo, x2 + xo, y2 + yo))
    water_mask = segment_water_land(img)
    return merge_boxes(boxes), water_mask

def draw_objects(image_path: str, objects, water_mask: np.ndarray, output_path: str):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    img = draw_water_contours(img, water_mask)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.imwrite(output_path, img)

def process_folder(input_folder: str, output_folder: str, max_tile_size=(2000, 2000),
                   low: float = 0.40, mid: float = 0.20, area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        pct_dark = image_dark_percent(inp) * 100
        scene = "water-heavy" if pct_dark > low * 100 else "mixed" if pct_dark > mid * 100 else "urban"
        print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")
        boxes, water_mask = process_image(inp, max_tile_size, low, mid, area_min)
        out_img = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, water_mask, out_img)
        box_df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
        box_df.to_csv(os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_boxes.csv"), index=False)
        summary.append({"image": fname, "dark_%": pct_dark, "objects": len(boxes),
                        "scene": scene, "output": out_img})
    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        print("CSV summary saved → summary.csv")




Намного лучше, но без выделения воды и с некоторыми косяками связанными с выделением точек интереса

In [ ]:
from __future__ import annotations
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd


def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    return float((img < dark_thresh).sum()) / img.size

def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)

def analyse_folder_darkness(folder: str,
                            dark_thresh: int = 40,
                            csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df


def preprocess_with_boost(img: np.ndarray) -> np.ndarray:
    img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_boost)
    return img_clahe


def tune_params(tile: np.ndarray,
                low: float = 0.40,
                mid: float = 0.20,
                area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:
        return area_min_urban / 5, 1.5
    if dr > mid:
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2


def merge_boxes(boxes: List[Tuple[int, int, int, int]], merge_distance: int = 10) -> List[Tuple[int, int, int, int]]:
    if not boxes:
        return []
    merged = []
    used = [False] * len(boxes)
    for i in range(len(boxes)):
        if used[i]:
            continue
        x1, y1, x2, y2 = boxes[i]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            x1_, y1_, x2_, y2_ = boxes[j]
            if x1_ <= x2 + merge_distance and x2_ >= x1 - merge_distance and \
               y1_ <= y2 + merge_distance and y2_ >= y1 - merge_distance:
                x1 = min(x1, x1_)
                y1 = min(y1, y1_)
                x2 = max(x2, x2_)
                y2 = max(y2, y2_)
                used[j] = True
        merged.append((x1, y1, x2, y2))
        used[i] = True
    return merged


def detect_objects_simple(img: np.ndarray,
                          area_thresh: float = 500,
                          min_mean_brightness: int = 40) -> List[Tuple[int, int, int, int]]:
    # Никакого буста
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)

    bin_img = cv2.adaptiveThreshold(img, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objs = []

    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh
        if area < area_thresh:
            continue
        mean_val = cv2.mean(img[y:y+bh, x:x+bw])[0]
        if mean_val < min_mean_brightness:
            continue
        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs, merge_distance=10)


def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles


def process_image(image_path: str,
                  max_tile_size=(2000, 2000),
                  low: float = 0.40,
                  mid: float = 0.20,
                  area_min: float = 0.001):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    boxes = []
    for tile, xo, yo in split_image(img, max_tile_size):
        area_t, ratio_t = tune_params(tile, low, mid, area_min)
        for x1, y1, x2, y2 in detect_objects(tile, area_t, ratio_t):
            boxes.append((x1 + xo, y1 + yo, x2 + xo, y2 + yo))
    return merge_boxes(boxes)


def draw_objects(image_path: str, objects, output_path: str):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        color = (0, 0, 255)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    cv2.imwrite(output_path, img)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   low: float = 0.40,
                   mid: float = 0.20,
                   area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        pct_dark = image_dark_percent(inp) * 100
        if pct_dark > low * 100:
            scene = "water-heavy"
        elif pct_dark > mid * 100:
            scene = "mixed"
        else:
            scene = "urban"
        print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")
        boxes = process_image(inp, max_tile_size, low, mid, area_min)
        out_img = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, out_img)

        box_df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
        box_df.to_csv(os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_boxes.csv"), index=False)

        summary.append({
            "image": fname,
            "dark_%": pct_dark,
            "objects": len(boxes),
            "scene": scene,
            "output": out_img
        })

    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        print("CSV summary saved → summary.csv")


# Пример запуска
# process_folder(input_folder="images", output_folder="output")

Что-то прям ебать какое хорошее 3 эпоха логика связана с тем что мы функцией preprocess_with_boost "Заливаем" фон изображения синим для контрастности точек инетерса и работаем исключительно по ним

In [91]:
from __future__ import annotations
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd


def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    return float((img < dark_thresh).sum()) / img.size

def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)

def analyse_folder_darkness(folder: str,
                            dark_thresh: int = 40,
                            csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df


def preprocess_with_boost(img: np.ndarray) -> np.ndarray:
    img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_boost)
    return img_clahe


def tune_params(tile: np.ndarray,
                low: float = 0.40,
                mid: float = 0.20,
                area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:
        return area_min_urban / 5, 1.5
    if dr > mid:
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2


def merge_boxes(boxes: List[Tuple[int, int, int, int]], merge_distance: int = 10) -> List[Tuple[int, int, int, int]]:
    if not boxes:
        return []
    merged = []
    used = [False] * len(boxes)
    for i in range(len(boxes)):
        if used[i]:
            continue
        x1, y1, x2, y2 = boxes[i]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            x1_, y1_, x2_, y2_ = boxes[j]
            if x1_ <= x2 + merge_distance and x2_ >= x1 - merge_distance and \
               y1_ <= y2 + merge_distance and y2_ >= y1 - merge_distance:
                x1 = min(x1, x1_)
                y1 = min(y1, y1_)
                x2 = max(x2, x2_)
                y2 = max(y2, y2_)
                used[j] = True
        merged.append((x1, y1, x2, y2))
        used[i] = True
    return merged

def detect_objects(tile: np.ndarray,
                   area_thresh: float,
                   intensity_ratio: float,
                   min_mean_brightness: int = 40) -> List[Tuple[int, int, int, int]]:
    img = preprocess_with_boost(tile)

    bin_img = cv2.adaptiveThreshold(img, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    objs = []
    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh
        if area < 400:
            continue
        aspect = max(bw / bh, bh / bw)
        if aspect > 15:
            continue
        compactness = area / (4 * np.pi * ((bw / 2 + bh / 2)**2))
        if compactness < 0.01:
            continue

        # Отсекаем совсем тёмные объекты по средней яркости
        mean_val = cv2.mean(img[y:y+bh, x:x+bw])[0]
        if mean_val < min_mean_brightness:
            continue

        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs)

def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles


def process_image(image_path: str,
                  max_tile_size=(2000, 2000),
                  low: float = 0.40,
                  mid: float = 0.20,
                  area_min: float = 0.001,
                  min_mean_brightness: int = 40):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    boxes = []
    for tile, xo, yo in split_image(img, max_tile_size):
        area_t, ratio_t = tune_params(tile, low, mid, area_min)
        for x1, y1, x2, y2 in detect_objects(tile, area_t, ratio_t, min_mean_brightness):
            boxes.append((x1 + xo, y1 + yo, x2 + xo, y2 + yo))
    return merge_boxes(boxes)


def detect_water_contours(img: np.ndarray, threshold: int = 50) -> List[np.ndarray]:
    """Возвращает контуры темных (водных) участков."""
    gray = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def draw_objects(image_path: str, objects, output_path: str):
    img_gray = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    img_color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    # Объекты интереса
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Контуры воды
    contours = detect_water_contours(img_gray)
    cv2.drawContours(img_color, contours, -1, (255, 0, 0), 1)

    cv2.imwrite(output_path, img_color)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   low: float = 0.40,
                   mid: float = 0.20,
                   area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        pct_dark = image_dark_percent(inp) * 100
        if pct_dark > low * 100:
            scene = "water-heavy"
        elif pct_dark > mid * 100:
            scene = "mixed"
        else:
            scene = "urban"
        print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")
        boxes = process_image(inp, max_tile_size, low, mid, area_min)
        out_img = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, out_img)

        box_df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
        box_df.to_csv(os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_boxes.csv"), index=False)

        summary.append({
            "image": fname,
            "dark_%": pct_dark,
            "objects": len(boxes),
            "scene": scene,
            "output": out_img
        })

    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        print("CSV summary saved → summary.csv")


# Пример запуска
# process_folder(input_folder="images", output_folder="output")

In [92]:
process_folder(
    input_folder="images",
    output_folder="output",
)

8.jpg: dark 63.9% → water-heavy params
9.jpg: dark 96.1% → water-heavy params
4.jpg: dark 34.6% → mixed params
5.jpg: dark 56.6% → water-heavy params
7.jpg: dark 31.3% → mixed params
6.jpg: dark 71.4% → water-heavy params
2.jpg: dark 61.4% → water-heavy params
3.jpg: dark 33.7% → mixed params
1.jpg: dark 68.9% → water-heavy params
CSV summary saved → summary.csv


Эпоха 4 - Развиваем историю Эпохи три, только меняем фон на фулл черный + добавляем работу двойной алгоритм, где мы работаем с заливкой а затем просто с обычным изображением, мержим все найденные точки и фиксируем прибыль, если алгос не нашел объект - я его маму шатал (Ниже два алгоса)

In [93]:
from __future__ import annotations
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd



def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    return float((img < dark_thresh).sum()) / img.size

def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)

def analyse_folder_darkness(folder: str,
                            dark_thresh: int = 40,
                            csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df

def preprocess_with_boost(img: np.ndarray) -> np.ndarray:
    # Усиление контраста
    img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)

    # CLAHE улучшает локальный контраст
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_boost)

    # Заливка тёмного в ноль
    _, img_black_bg = cv2.threshold(img_clahe, 60, 255, cv2.THRESH_TOZERO)
    # img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    # img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)
    # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # img_clahe = clahe.apply(img_boost)

    # ⚠️ Сохраняем для отладки (можно убрать позже)
    cv2.imwrite("debug_preprocessed.png", img_clahe)

    return img_clahe

def tune_params(tile: np.ndarray,
                low: float = 0.40,
                mid: float = 0.20,
                area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:
        return area_min_urban / 5, 1.5
    if dr > mid:
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2


def merge_boxes(boxes: List[Tuple[int, int, int, int]], merge_distance: int = 10) -> List[Tuple[int, int, int, int]]:
    if not boxes:
        return []
    merged = []
    used = [False] * len(boxes)
    for i in range(len(boxes)):
        if used[i]:
            continue
        x1, y1, x2, y2 = boxes[i]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            x1_, y1_, x2_, y2_ = boxes[j]
            if x1_ <= x2 + merge_distance and x2_ >= x1 - merge_distance and \
               y1_ <= y2 + merge_distance and y2_ >= y1 - merge_distance:
                x1 = min(x1, x1_)
                y1 = min(y1, y1_)
                x2 = max(x2, x2_)
                y2 = max(y2, y2_)
                used[j] = True
        merged.append((x1, y1, x2, y2))
        used[i] = True
    return merged

def detect_objects(tile: np.ndarray,
                   area_thresh: float,
                   intensity_ratio: float,
                   min_mean_brightness: int = 40) -> List[Tuple[int, int, int, int]]:
    img = preprocess_with_boost(tile)

    bin_img = cv2.adaptiveThreshold(img, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    objs = []
    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh
        if area < 400:
            continue
        aspect = max(bw / bh, bh / bw)
        if aspect > 15:
            continue
        compactness = area / (4 * np.pi * ((bw / 2 + bh / 2)**2))
        if compactness < 0.01:
            continue

        # Отсекаем совсем тёмные объекты по средней яркости
        mean_val = cv2.mean(img[y:y+bh, x:x+bw])[0]
        if mean_val < min_mean_brightness:
            continue

        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs)

def detect_objects_simple(img: np.ndarray,
                          area_thresh: float = 500,
                          min_mean_brightness: int = 40) -> List[Tuple[int, int, int, int]]:
    # Никакого буста
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)

    bin_img = cv2.adaptiveThreshold(img, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objs = []

    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh
        if area < area_thresh:
            continue
        mean_val = cv2.mean(img[y:y+bh, x:x+bw])[0]
        if mean_val < min_mean_brightness:
            continue
        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs, merge_distance=10)


def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles


def process_image_combined(image_path: str,
                           area_min: float = 0.001,
                           min_mean_brightness: int = 40):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    
    h, w = img.shape
    area_thresh = h * w * area_min

    # Старый метод
    area_t, ratio_t = tune_params(img)
    boxes_a = detect_objects(img, area_t, ratio_t, min_mean_brightness)

    # Новый метод без тюна
    boxes_b = detect_objects_simple(img, area_thresh, min_mean_brightness)

    # Объединяем
    merged = merge_boxes(boxes_a + boxes_b, merge_distance=15)

    return merged



def detect_water_contours(img: np.ndarray, threshold: int = 50) -> List[np.ndarray]:
    """Возвращает контуры темных (водных) участков."""
    gray = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def draw_objects(image_path: str, objects, output_path: str):
    img_gray = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    img_color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    # Объекты интереса
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Контуры воды
    contours = detect_water_contours(img_gray)
    cv2.drawContours(img_color, contours, -1, (255, 0, 0), 1)

    cv2.imwrite(output_path, img_color)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   low: float = 0.40,
                   mid: float = 0.20,
                   area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        pct_dark = image_dark_percent(inp) * 100
        if pct_dark > low * 100:
            scene = "water-heavy"
        elif pct_dark > mid * 100:
            scene = "mixed"
        else:
            scene = "urban"
        print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")
        boxes = boxes = process_image_combined(inp, area_min=area_min)
        
        out_img = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, out_img)

        box_df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
        box_df.to_csv(os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_boxes.csv"), index=False)

        summary.append({
            "image": fname,
            "dark_%": pct_dark,
            "objects": len(boxes),
            "scene": scene,
            "output": out_img
        })

    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        print("CSV summary saved → summary.csv")


# Пример запуска
# process_folder(input_folder="images", output_folder="output")

In [94]:
process_folder(
    input_folder="images",
    output_folder="output",
)

8.jpg: dark 63.9% → water-heavy params
9.jpg: dark 96.1% → water-heavy params
4.jpg: dark 34.6% → mixed params
5.jpg: dark 56.6% → water-heavy params
7.jpg: dark 31.3% → mixed params
6.jpg: dark 71.4% → water-heavy params
2.jpg: dark 61.4% → water-heavy params
3.jpg: dark 33.7% → mixed params
1.jpg: dark 68.9% → water-heavy params
CSV summary saved → summary.csv


In [95]:
from __future__ import annotations
import os
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd


def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    return float((img < dark_thresh).sum()) / img.size

def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)

def analyse_folder_darkness(folder: str,
                            dark_thresh: int = 40,
                            csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df

def preprocess_with_boost(img: np.ndarray) -> np.ndarray:
    # Усиление контраста
    img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)

    # CLAHE улучшает локальный контраст
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_boost)

    # Заливка тёмного в ноль
    _, img_black_bg = cv2.threshold(img_clahe, 60, 255, cv2.THRESH_TOZERO)
    # img_boost = cv2.normalize(img, None, 255, 0, cv2.NORM_MINMAX)
    # img_boost = cv2.convertScaleAbs(img_boost, alpha=2.0, beta=50)
    # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # img_clahe = clahe.apply(img_boost)

    # ⚠️ Сохраняем для отладки (можно убрать позже)
    cv2.imwrite("debug_preprocessed.png", img_clahe)

    return img_clahe

def tune_params(tile: np.ndarray,
                low: float = 0.40,
                mid: float = 0.20,
                area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:
        return area_min_urban / 5, 1.5
    if dr > mid:
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2


def merge_boxes(boxes: List[Tuple[int, int, int, int]], merge_distance: int = 10) -> List[Tuple[int, int, int, int]]:
    if not boxes:
        return []
    merged = []
    used = [False] * len(boxes)
    for i in range(len(boxes)):
        if used[i]:
            continue
        x1, y1, x2, y2 = boxes[i]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            x1_, y1_, x2_, y2_ = boxes[j]
            if x1_ <= x2 + merge_distance and x2_ >= x1 - merge_distance and \
               y1_ <= y2 + merge_distance and y2_ >= y1 - merge_distance:
                x1 = min(x1, x1_)
                y1 = min(y1, y1_)
                x2 = max(x2, x2_)
                y2 = max(y2, y2_)
                used[j] = True
        merged.append((x1, y1, x2, y2))
        used[i] = True
    return merged

def detect_objects(tile: np.ndarray,
                   area_thresh: float,
                   intensity_ratio: float,
                   min_mean_brightness: int = 40) -> List[Tuple[int, int, int, int]]:
    img = preprocess_with_boost(tile)

    bin_img = cv2.adaptiveThreshold(img, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 31, -5)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k)

    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    objs = []
    for cnt in contours:
        x, y, bw, bh = cv2.boundingRect(cnt)
        area = bw * bh
        if area < 400:
            continue
        aspect = max(bw / bh, bh / bw)
        if aspect > 15:
            continue
        compactness = area / (4 * np.pi * ((bw / 2 + bh / 2)**2))
        if compactness < 0.01:
            continue

        # Отсекаем совсем тёмные объекты по средней яркости
        mean_val = cv2.mean(img[y:y+bh, x:x+bw])[0]
        if mean_val < min_mean_brightness:
            continue

        objs.append((x, y, x + bw, y + bh))

    return merge_boxes(objs)

def detect_objects_advanced(tile: np.ndarray,
                            area_thresh: float = 0.001,
                            intensity_ratio: float = 2.0):
    pre = preprocess(tile)

    # adaptive threshold tuned for speckle
    bin_img = cv2.adaptiveThreshold(pre, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 51, -5)

    # morphology to remove pepper noise and fill holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel, iterations=1)
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, kernel, iterations=2)

    labeled, num = label(bin_img)
    objs = []
    min_pixels = int(area_thresh * tile.shape[0] * tile.shape[1])
    background_median = np.median(tile)

    for i in range(1, num + 1):
        ys, xs = np.where(labeled == i)
        if xs.size < min_pixels:
            continue
        if tile[ys, xs].mean() < intensity_ratio * background_median:
            continue
        x1, x2 = xs.min(), xs.max()
        y1, y2 = ys.min(), ys.max()
        objs.append((x1, y1, x2, y2))
    return objs


def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles


def process_image_combined(image_path: str,
                           area_min: float = 0.001,
                           min_mean_brightness: int = 40):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    
    h, w = img.shape
    area_thresh = h * w * area_min

    # Старый метод
    area_t, ratio_t = tune_params(img)
    boxes_a = detect_objects(img, area_t, ratio_t, min_mean_brightness)

    # Новый метод без тюна
    boxes_b = detect_objects_simple(img, area_thresh, min_mean_brightness)

    # Объединяем
    merged = merge_boxes(boxes_a + boxes_b, merge_distance=15)

    return merged



def detect_water_contours(img: np.ndarray, threshold: int = 50) -> List[np.ndarray]:
    """Возвращает контуры темных (водных) участков."""
    gray = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY_INV)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def draw_objects(image_path: str, objects, output_path: str):
    img_gray = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    img_color = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    # Объекты интереса
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # Контуры воды
    contours = detect_water_contours(img_gray)
    cv2.drawContours(img_color, contours, -1, (255, 0, 0), 1)

    cv2.imwrite(output_path, img_color)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   low: float = 0.40,
                   mid: float = 0.20,
                   area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        pct_dark = image_dark_percent(inp) * 100
        if pct_dark > low * 100:
            scene = "water-heavy"
        elif pct_dark > mid * 100:
            scene = "mixed"
        else:
            scene = "urban"
        ##print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")
        boxes = boxes = process_image_combined(inp, area_min=area_min)
        
        out_img = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, out_img)

        box_df = pd.DataFrame(boxes, columns=['x1', 'y1', 'x2', 'y2'])
        box_df.to_csv(os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_boxes.csv"), index=False)

        summary.append({
            "image": fname,
            "dark_%": pct_dark,
            "objects": len(boxes),
            "scene": scene,
            "output": out_img
        })

    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        ##print("CSV summary saved → summary.csv")


# Пример запуска
process_folder(input_folder="images", output_folder="output")